In [1]:
#Intellisense

%config IPCompleter.greedy=True

In [3]:
import pandas as pd
#class to listen the tweets based on certain keywords/ hastags
from tweepy.streaming import StreamListener
# class for authenticating
from tweepy import OAuthHandler
from tweepy import Stream

In [ ]:
#access project via service account
PROJECT_ID = 'rgadata-m11'

#create a client instance for you project
from google.cloud import bigquery
from google.oauth2 import service_account
creds = service_account.Credentials.from_service_account_file(r"C:\Users\varunn\Documents\kaggle-competiton\credentials\credentials.json")
client = bigquery.Client(project=PROJECT_ID,location='US',credentials=creds)


In [11]:
#access project via user account
from google_auth_oauthlib import flow

# TODO: Uncomment the line below to set the `launch_browser` variable.
launch_browser = True
#
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    r"C:\Users\varunn\Documents\kaggle-competiton\credentials\credentials.json",
    scopes=['https://www.googleapis.com/auth/bigquery'])

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=275664291874-jutmedsp4bmt6o40tm0809phnk9tnruc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=hplCXwb3v9coXk3h2IBIlk2z0ZSFaY&access_type=offline


In [ ]:
client = bigquery.Client(project=PROJECT_ID,location='US',credentials=credentials)

In [13]:
#create a new dataset
client.create_dataset('sandboxm11')

Dataset(DatasetReference('rgadata-m11', 'sandboxm11'))

In [ ]:
#create a table in dataset
client.create_table(f"{PROJECT_ID}.sandboxm11.adspend")

In [15]:
import pandas as pd

fd = (r"C:\Users\varunn\Documents\Confluence attachments\MarketingSci Regression analysis\adspendraw\merged_df.xlsx")
adspend = pd.read_excel(fd,sheet_name=0)

In [16]:
adspend.columns = adspend.iloc[0,:]
adspend = adspend.drop(0)
adspend = adspend.fillna(0)
adspend = adspend.rename(columns = {adspend.columns[0]:'first'})
adspend = adspend.drop('first', axis=1)

In [17]:
adspend = adspend.iloc[:,0:11]

In [18]:
adspend.columns

Index(['MEDIA', 'INDUSTRY', 'JAN 2000  DOLS (000)', 'FEB 2000  DOLS (000)',
       'MAR 2000  DOLS (000)', 'APR 2000  DOLS (000)', 'MAY 2000  DOLS (000)',
       'JUN 2000  DOLS (000)', 'JUL 2000  DOLS (000)', 'AUG 2000  DOLS (000)',
       'SEP 2000  DOLS (000)'],
      dtype='object', name=0)

In [19]:
adspend = adspend.rename(columns={'JAN 2000  DOLS (000)':'Jan2000','FEB 2000  DOLS (000)':'Feb2000','MAR 2000  DOLS (000)':'mar2000', 'APR 2000  DOLS (000)':'apr2000', 'MAY 2000  DOLS (000)':'may2000',
       'JUN 2000  DOLS (000)':'jun2000', 'JUL 2000  DOLS (000)':'jul2000', 'AUG 2000  DOLS (000)':'aug2000',
       'SEP 2000  DOLS (000)':'sep2000'})

In [20]:
dataset_id = 'sandboxm11'
table_id = 'adspend'

#tell client everything it needs to know to upload your data
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig(schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("MEDIA", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("INDUSTRY", bigquery.enums.SqlTypeNames.STRING),
    ],)
job_config.autodetect = True

job = client.load_table_from_dataframe(adspend,table_ref,job_config=job_config)
job.result()
print ("loaded {} rows into {}:{}".format(job.output_rows,dataset_id,table_id))

loaded 1123 rows into sandboxm11:adspend


In [ ]:
#query the data
query = f""" Select * from {PROJECT_ID}.sandboxm11.adspend limit 10"""
query_job = client.query(query)
#data = query_job.to_dataframe()
query_job

In [28]:

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, r'C:\Users\varunn\Documents\kaggle-competiton\credentials')

import twittercreds


In [144]:
import tweepy
from twitter import *

auth = tweepy.OAuthHandler(twittercreds.CONSUMER_KEY,twittercreds.CONSUMER_SECRET)
auth.set_access_token(twittercreds.ACCESS_TOKEN,twittercreds.ACCESS_TOKEN_SECRET)


api = tweepy.API(auth)

public_tweets = api.home_timeline()

# retrieve global trends.
# other localised trends can be specified by looking up WOE IDs:
#   http://developer.yahoo.com/geo/geoplanet/
# twitter API docs: https://dev.twitter.com/rest/reference/get/trends/place
#-----------------------------------------------------------------------
#trends = api.trends_available()
trends = api.trends_place(id =23424935, exclude = 'hashtags')
for tweet in trends:
    print(tweet)

{'trends': [{'name': 'COVID', 'url': 'http://twitter.com/search?q=COVID', 'promoted_content': None, 'query': 'COVID', 'tweet_volume': 3039769}, {'name': 'san juan', 'url': 'http://twitter.com/search?q=%22san+juan%22', 'promoted_content': None, 'query': '%22san+juan%22', 'tweet_volume': 22608}, {'name': 'Monat', 'url': 'http://twitter.com/search?q=Monat', 'promoted_content': None, 'query': 'Monat', 'tweet_volume': None}, {'name': 'Malcolm X', 'url': 'http://twitter.com/search?q=%22Malcolm+X%22', 'promoted_content': None, 'query': '%22Malcolm+X%22', 'tweet_volume': 61469}, {'name': 'Andy Halliday', 'url': 'http://twitter.com/search?q=%22Andy+Halliday%22', 'promoted_content': None, 'query': '%22Andy+Halliday%22', 'tweet_volume': None}, {'name': 'Annie Glenn', 'url': 'http://twitter.com/search?q=%22Annie+Glenn%22', 'promoted_content': None, 'query': '%22Annie+Glenn%22', 'tweet_volume': 11361}, {'name': 'Ravi Zacharias', 'url': 'http://twitter.com/search?q=%22Ravi+Zacharias%22', 'promoted_c

In [ ]:
#class to print the tweets

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(twittercreds.CONSUMER_KEY, twittercreds.CONSUMER_SECRET)
        auth.set_access_token(twittercreds.ACCESS_TOKEN, twittercreds.ACCESS_TOKEN_SECRET)
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            #with open(self.fetched_tweets_filename, 'a') as tf:
            #    tf.write(data)
            self.fetched_tweets_filename = data
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          

    def on_error(self, status):
        print(status)

 
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["testing twitter api"]
    fetched_tweets_filename = {}

    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

In [1]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials
from searchtweets import collect_results


In [4]:
#permim setup credentials
 
premium_search_args = load_credentials("C:/Users/varunn/Documents/kaggle-competiton/credentials/twitter_keys",
                                       yaml_key="search_tweets_premium",
                                       env_overwrite=False)


c:\users\varunn\appdata\local\continuum\anaconda3\envs\py37\lib\site-packages\searchtweets\credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


In [104]:
#query search criteria
rule = gen_rule_payload("covid19", results_per_call=100) # testing with a sandbox account
#rule = gen_rule_payload("This Indian paint manufacturer is boosting salaries amid the Covid-19 crisis", results_per_call=100) # testing with a sandbox account
print(rule)



{"query": "covid19", "maxResults": 100}


In [ ]:
tweets1 = collect_results(rule,max_results=4000,result_stream_args=premium_search_args)

In [114]:
tweets1[990]

{'created_at': 'Tue May 26 14:38:43 +0000 2020',
 'id': 1265291277408632832,
 'id_str': '1265291277408632832',
 'text': 'RT @mbachelet: Saludo la legalización del matrimonio civil igualitario en Costa Rica 🇨🇷 \nTodos nacemos libres e iguales y tenemos derecho a…',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 32647519,
  'id_str': '32647519',
  'name': 'Cristobal Acevedo',
  'screen_name': 'elcristobalac',
  'location': 'Linares - Santiago',
  'url': None,
  'description': '#RenunciaPiñera #Apruebo\n http://instagram.com/perocristobalac',
  'translator_type': 'regular',
  'protected': False,
  'verified': False,
  'followers_count': 1517,
  'friends_count': 1832,
  'listed_count': 20,
  'favourites_count': 13442,
  'statuses_count': 7

In [39]:
[print(tweetd.text, end='\n\n') for tweetd in tweets1]

RT @fanniew08: Continue to honor #MLKDay #MLK 
#MLK2020 
#VoterRegistrationCheck 
#VoterSuppressionIsReal
Support:
@fairfightaction 
@MomsD…

RT @IAmRichBenjamin: WH doing damnedest to dodge the Law. FBI arrests 3 Neo-Nazis from military-style training camps headed to gun rally in…

RT @erinlimofficial: #MLK2020 https://t.co/v4aWcXCnkD

RT @HawaiiDelilah: Powerful ad from Joe Biden - the one candidate continuously talking about the poisonous hatred Trump disseminates. #MLK2…

RT @HawaiiDelilah: Powerful ad from Joe Biden - the one candidate continuously talking about the poisonous hatred Trump disseminates. #MLK2…

RT @quotesbybaldwin: and baffle them.  He does not offer any easy comfort and this keeps his hearers absolutely tense. He allows them their…

These drive thru graduations are SUPER BOMB.. I know it was not ideal, but y’all’s graduation ceremonies were condu… https://t.co/pQXlpoakYi

RT @rankandfileca: "The labor movement was the principal force that transformed misery and de

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]